#### Import some libraries

In [1]:
import numpy as np
import time
import cv2
import os
import glob
import matplotlib.pyplot as plt

#### Load yolo weights and cfg 

In [5]:
weights_path = os.path.join ("yolo","yolov3.weights")
config_path = os.path.join ("yolo","yolov3.cfg")

print("Loaded Yolo weights and config")

Loaded Yolo weights and config
Loaded Yolo weights and config


#### Load the neural net in the cv2

In [6]:
net = cv2.dnn.readNetFromDarknet(config_path , weights_path)

#### Get Layer Names

In [7]:
names = net.getLayerNames()

#### Load test image

In [13]:
image_path = os.path.join ("yolo","test1.jpg")
img=cv2.imread(image_path)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 

In [20]:
(H,W)= img.shape[:2]
# layers_names = [names[i[0]-1] for i in net.getUnconnectedOutLayers()]
layers_names = [names[i - 1] for i in net.getUnconnectedOutLayers()]

In [21]:
layers_names

['yolo_82', 'yolo_94', 'yolo_106']

['yolo_82', 'yolo_94', 'yolo_106']

#### Run the inference on the test image

In [23]:
blob = cv2.dnn.blobFromImage (img, 1/255.0 , (416,416), crop=False,swapRB=False)
net.setInput(blob)
#calculate the runtime of the algorithm
start_t=time.time()
layers_output=net.forward(layers_names)
print ("A forward pass through yolov3 took {}".format(time.time()- start_t))

A forward pass through yolov3 took 0.7616486549377441
A forward pass through yolov3 took 0.7616486549377441


In [24]:
layers_output[0].shape

(507, 85)

(507, 85)

In [27]:
boxes =[]
confidences=[]
classIDs=[]

In [28]:
for output in layers_output:
    for detection in output:
        scores=detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        
        if confidence > 0.85:
            box = detection[:4]*np.array([W,H,W,H])
            bx, by, bw, bh = box.astype("int")
            
            x=int(bx - (bw/2))
            y=int(by - (bh/2))
            
            boxes.append([x,y,int(bw),int(bh)])
            confidences.append(float(confidence))
            classIDs.append(classID)

In [30]:
idxs = cv2.dnn.NMSBoxes(boxes,confidences,0.8,0.8)

In [34]:
labels_path = os.path.join("yolo","coco.names")
labels = open(labels_path).read().strip().split("\n")

#### Plot the bounding boxes in the image

In [44]:
for i in idxs.flatten():
    (x,y) = [boxes[i][0],boxes[i][1]]
    (w,h)=[boxes[i][2],boxes[i][3]]
    
    cv2.rectangle(img, (x,y),(x+w,y+h),(255,0,0),2)
    cv2.putText (img,"{}:{}".format(labels[classIDs[i]],confidences[i]),(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)
                 
# plt.imshow(img)
cv2.imshow("Images",cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)

-1

-1